In [124]:
import random
import enum
class WordleOutput(enum.Enum):
    invalid_guess = "Invalid Guess"
    game_over = "Game Over"
    win = "win"


class WordleGame:
    """A class to manage a wordle game"""
    word_file = "/Users/noahpursell/Documents/Programming/Python/WordleGame/words.txt"
    file = open(word_file, "r")
    words = file.read().split("\n")
    
    def __init__(self):
        # Load words
        
        self.max_guesses = 6
        self.word_length = 5
    
    def start_game(self):
        """Start a wordle game"""
        self.target_word = self.words[random.randint(0, len(self.words))]
        self.guesses = 0
        self.game_history = []
    
    def make_guess(self, guess: str):
        if self.target_word == guess:
            return WordleOutput.win # Win condition
        if (self.guesses < self.max_guesses): # Hasnt used all guesses
            if guess in self.words and len(guess) == self.word_length:
                # Valid word
                output_list = [[letter, 0] for letter in guess]
                # 0 means no info, 1 means that letter is in the target word, 2 means that is a parallel letter
                guess_letter_list = list(guess)
                target_word_letter_list = list(self.target_word)

                # Check for parallel letters
                for i in range(self.word_length):
                    if(guess_letter_list[i] == target_word_letter_list[i]):
                        output_list[i][1] = 2
                        target_word_letter_list[i] = "CHECKED"
                # Check for non-parallel same letters
                for i in range(self.word_length):
                    if output_list[i][1] != 2: # If the letter has not been identified as a prallel letter
                        for x in range(self.word_length):
                            if guess_letter_list[i] == target_word_letter_list[x]: # If the letter is in both words
                                output_list[i][1] = 1 
                                target_word_letter_list[x] = "CHECKED"
                self.guesses += 1
                self.game_history.append(output_list)
                return output_list
            else:
                return WordleOutput.invalid_guess
        else:
            return WordleOutput.game_over
    
    def check_guess(self, guess, target_word):
        if guess in self.words and len(guess) == self.word_length:
            # Valid word
            output_list = [[letter, 0] for letter in guess]
            # 0 means no info, 1 means that letter is in the target word, 2 means that is a parallel letter
            guess_letter_list = list(guess)
            target_word_letter_list = list(target_word)

            # Check for parallel letters
            for i in range(self.word_length):
                if(guess_letter_list[i] == target_word_letter_list[i]):
                    output_list[i][1] = 2
                    target_word_letter_list[i] = "CHECKED"
            # Check for non-parallel same letters
            for i in range(self.word_length):
                if output_list[i][1] != 2: # If the letter has not been identified as a prallel letter
                    for x in range(self.word_length):
                        if guess_letter_list[i] == target_word_letter_list[x]: # If the letter is in both words
                            output_list[i][1] = 1 
                            target_word_letter_list[x] = "CHECKED"
            return output_list


In [169]:
from collections import defaultdict
import math

class WordleGamePlayer:
    """A class to manage an auto wordle player"""
    def __init__(self):
        """Create a wordle game player"""
        self.wordleGame = WordleGame()
    
    def play_game(self):
        self.wordleGame.start_game()
        output = None
        while output != WordleOutput.game_over and output != WordleOutput.win:
            output = self.wordleGame.make_guess(input("Enter your word"))
            print(output)
    
    def number_of_combimations_from_word(self, word, word_list):
        """Returns a dictionay containing every possible rating of the input word, and all the words that fit that rating"""
        game = WordleGame()
        combo_dict = defaultdict(list)
        for target_word in word_list:
            combo_dict[tuple([score for letter, score in game.check_guess(word, target_word)])].append(target_word)
        
        return combo_dict

    def get_expected_return(self, word, word_list):
        """Returns the expected return of a word, where the expected return is the number of bits of information"""
        # This is calculated by taking the sum(prob(outcome) * amount of info in that outcome)
        combo_dict = self.number_of_combimations_from_word(word, word_list)
        sum = 0
        for key, value in combo_dict.items():
            p = len(value) / float(len(word_list))
            info = math.log(1 / p, 2)
            sum += p * info
            
        return sum
    
    def play_game_automatically(self):
        word_list = WordleGame.words
        count = 0
        # Find best move
        word_values = []
        for word in word_list:
            count += 1
            if count % 100 == 0:
                print(count)
            value = self.get_expected_return(word, word_list)
            word_values.append((word, value))
        best_word = max(word_values, key=lambda value: value[1])
        print(best_word)



In [ ]:
player = WordleGamePlayer()
player.play_game_automatically()

In [206]:
def permutations(length, possible_values):
    num_perms = possible_values ** length
    permutations = [[0 for x in range(length)]]
    for i in range(num_perms):
        perm = list(permutations[-1])

        for x in range(length):
            if (i % (possible_values ** x) == 0):
                # Increment
                if perm[x] == possible_values - 1:
                    perm[x] = 0
                else:
                    perm[x] += 1
        permutations.append(perm)
    
    return permutations
    
        

In [208]:
print(permutations(3, 3))

[1, 1, 1]
[2, 1, 1]
[0, 1, 1]
[1, 2, 1]
[2, 2, 1]
[0, 2, 1]
[1, 0, 1]
[2, 0, 1]
[0, 0, 1]
[1, 1, 2]
[2, 1, 2]
[0, 1, 2]
[1, 2, 2]
[2, 2, 2]
[0, 2, 2]
[1, 0, 2]
[2, 0, 2]
[0, 0, 2]
[1, 1, 0]
[2, 1, 0]
[0, 1, 0]
[1, 2, 0]
[2, 2, 0]
[0, 2, 0]
[1, 0, 0]
[2, 0, 0]
[0, 0, 0]
[[0, 0, 0], [1, 1, 1], [2, 1, 1], [0, 1, 1], [1, 2, 1], [2, 2, 1], [0, 2, 1], [1, 0, 1], [2, 0, 1], [0, 0, 1], [1, 1, 2], [2, 1, 2], [0, 1, 2], [1, 2, 2], [2, 2, 2], [0, 2, 2], [1, 0, 2], [2, 0, 2], [0, 0, 2], [1, 1, 0], [2, 1, 0], [0, 1, 0], [1, 2, 0], [2, 2, 0], [0, 2, 0], [1, 0, 0], [2, 0, 0], [0, 0, 0]]
